In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
volume_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/'

In [4]:
volume1 = np.load(os.path.join(volume_dir, 'volume_MD589.npz'))['arr_0']
nx1, ny1, nz1 = volume1.shape
print nx1, ny1, nz1

IOError: [Errno 2] No such file or directory: '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/volume_MD589.npz'

In [ ]:
volume2 = np.load(os.path.join(volume_dir, 'volume_MD594.npz'))['arr_0']
nx2, ny2, nz2 = volume2.shape
print nx2, ny2, nz2

In [8]:
# canvas2 = np.zeros((2000,2500,1500), np.int8)
# canvas2_center = (canvas2.shape[0]/2, canvas2.shape[1]/2, canvas2.shape[2]/2)
# canvas2[canvas2_center[0]-nx2/2:canvas2_center[0]+(nx2-nx2/2), 
#        canvas2_center[1]-ny2/2:canvas2_center[1]+(ny2-ny2/2), 
#        canvas2_center[2]-nz2/2:canvas2_center[2]+(nz2-nz2/2)] = volume2

In [38]:
canvas1 = np.zeros((1000,2000,1000), np.int8)
canvas1_center = (canvas1.shape[0]/2, canvas1.shape[1]/2, canvas1.shape[2]/2)
canvas1[canvas1_center[0]-nx1/2:canvas1_center[0]+(nx1-nx1/2), 
       canvas1_center[1]-ny1/2:canvas1_center[1]+(ny1-ny1/2), 
       canvas1_center[2]-nz1/2:canvas1_center[2]+(nz1-nz1/2)] = volume1

In [40]:
x1s_nz, y1s_nz, z1s_nz = np.where(canvas1)

In [41]:
print x1s_nz.min(), x1s_nz.max(), y1s_nz.min(), y1s_nz.max(), z1s_nz.min(), z1s_nz.max()

178 821 256 1744 139 861


In [45]:
centroid1 = (x1s_nz.mean(), y1s_nz.mean(), z1s_nz.mean())
print centroid1

(577.36733618629978, 759.18129272541194, 483.05166324577505)


In [42]:
x2s, y2s, z2s = np.where(volume2)

In [46]:
centroid2 = (x2s.mean(), y2s.mean(), z2s.mean())
print centroid2

(379.77627564048566, 546.81517678475086, 30.169603214171616)


In [8]:
print x2s.min(), x2s.max(), y2s.min(), y2s.max(), z2s.min(), z2s.max()

0 703 0 1378 0 53


In [9]:
from itertools import product

In [10]:
# C = canvas2[x2s, y2s, z2s]
C = volume2[x2s, y2s, z2s]

In [11]:
P = np.c_[x2s, y2s, z2s, np.ones((x2s.shape[0],))].T

In [106]:
def f(tx, ty, tz, theta_z, theta_y):
    
    cos_z = np.cos(theta_z)
    sin_z = np.sin(theta_z)
    cos_y = np.cos(theta_y)
    sin_y = np.sin(theta_y)
    Rz = np.array([[cos_z, -sin_z, 0], [sin_z, cos_z, 0], [0,0,1]])
    Ry = np.array([[cos_y, 0, -sin_y], [0, 1, 0], [sin_y,0,cos_y]])
    R = np.dot(Rz, Ry)    
    
    T = np.c_[R, [tx,ty,tz]]
#     T = np.c_[R, np.dot(R, [tx,ty,tz])]

    x1s, y1s, z1s = np.dot(T, P).astype(np.int)
    
    try:
        s = np.count_nonzero(C == canvas1[x1s, y1s, z1s])
    except IndexError:
        return 0
        
    return s

In [107]:
from joblib import Parallel, delayed

In [ ]:
init_tx = centroid1[0] - centroid2[0]
init_ty = centroid1[1] - centroid2[1]
init_tz = centroid1[2] - centroid2[2]

In [115]:
tx_grid = np.linspace(init_tx-400,init_tx+400,10)
ty_grid = np.linspace(init_ty-400,init_ty+400,10)
tz_grid = np.linspace(init_tz-200,init_tz+200,10)
theta_z_grid = np.deg2rad(np.linspace(-20,20,5))
theta_y_grid = np.deg2rad(np.linspace(-20,20,5))

In [108]:
n = 1000

tx_grid = np.random.randint(init_tx-400, init_tx+400, n)
ty_grid = np.random.randint(init_ty-400, init_ty+400, n)
tz_grid = np.random.randint(init_tz-200, init_tz+200, n)
# theta_z_grid = [0 for _ in range(n)]
# theta_y_grid = [0 for _ in range(n)]
theta_z_grid = np.deg2rad(np.random.randint(-20, 20, n))
theta_y_grid = np.deg2rad(np.random.randint(-20, 20, n))

In [116]:
import time
t = time.time()

# scores = Parallel(n_jobs=16)(delayed(f)(tx, ty, tz, theta_z, theta_y ) 
#              for tx, ty, tz, theta_z, theta_y in zip(tx_grid, ty_grid, tz_grid, theta_z_grid, theta_y_grid))
# print np.max(scores)
# print zip(tx_grid, ty_grid, tz_grid, np.rad2deg(theta_z_grid), np.rad2deg(theta_y_grid))[np.argmax(scores)]

scores = Parallel(n_jobs=16)(delayed(f)(tx, ty, tz, theta_z, theta_y ) 
             for tx, ty, tz, theta_z, theta_y in product(tx_grid, ty_grid, tz_grid, theta_z_grid, theta_y_grid))
xi, yi, zi = np.unravel_index(np.argmax(scores_mat), scores_mat.shape)
print tx_grid[xi], ty_grid[yi], tz_grid[zi]

print time.time() - t

NameError: name 'scores_mat' is not defined

In [96]:
import time
t = time.time()

scores = Parallel(n_jobs=16)(delayed(f)(tx, ty, tz, theta_z, theta_y ) 
             for tx, ty, tz, theta_z, theta_y in zip(tx_grid, ty_grid, tz_grid, theta_z_grid, theta_y_grid))

print time.time() - t

print np.max(scores)
print zip(tx_grid, ty_grid, tz_grid, np.rad2deg(theta_z_grid), np.rad2deg(theta_y_grid))[np.argmax(scores)]

27.6544790268
274177
(69, 421, 330, -9.0, -13.0)


In [69]:
scores = []

# for tx, ty, tz, theta_z, theta_y in product(tx_grid, ty_grid, tz_grid, theta_z_grid, theta_y_grid):
for tx, ty, tz, theta_z, theta_y in zip(tx_grid, ty_grid, tz_grid, theta_z_grid, theta_y_grid):
    
    import time
    t = time.time()
    
    print tx, ty, tz, theta_z, theta_y
    
    cos_z = np.cos(theta_z)
    sin_z = np.sin(theta_z)
    cos_y = np.cos(theta_y)
    sin_y = np.sin(theta_y)
    Rz = np.array([[cos_z, -sin_z, 0], [sin_z, cos_z, 0], [0,0,1]])
    Ry = np.array([[cos_y, 0, -sin_y], [0, 1, 0], [sin_y,0,cos_y]])
    R = np.dot(Rz, Ry)    
    
    T = np.c_[R, [tx,ty,tz]]
    x1s, y1s, z1s = np.dot(T, P).astype(np.int)
    
    print x1s.min(), x1s.max(), y1s.min(), y1s.max(), z1s.min(), z1s.max()
    
    try:
        s = np.count_nonzero(C == canvas1[x1s, y1s, z1s])
        print s
    except IndexError:
        s = 0
    
    scores.append(s) 
    
    print time.time() - t

100 212 452.882060032 0 0
100 803 212 1590 452 505
868
0.37620806694
94 358 452.882060032 0 0
94 797 358 1736 452 505
0
0.375512838364
101 318 452.882060032 0 0
101 804 318 1696 452 505
0
0.375268936157
258 239 452.882060032 0 0
258 961 239 1617 452 505
201
0.37552690506
268 114 452.882060032 0 0
268 971 114 1492 452 505
126
0.375246047974
153 314 452.882060032 0 0
153 856 314 1692 452 505
42
0.375594139099
240 182 452.882060032 0 0
240 943 182 1560 452 505
462
0.375012159348
341 165 452.882060032 0 0
341 1044 165 1543 452 505
0.280470132828
42 129 452.882060032 0 0
42 745 129 1507 452 505
1293
0.375720977783
45 157 452.882060032 0 0
45 748 157 1535 452 505
851
0.375609874725


In [ ]:
xi, yi, zi = np.unravel_index(np.argmax(scores_mat), scores_mat.shape)
tx_grid[xi], ty_grid[yi], tz_grid[zi]

In [ ]:
scores_mat = np.reshape(scores, (len(tx_grid), len(ty_grid), len(tz_grid)))
plt.matshow(scores_mat[:,0]);
plt.colorbar();
plt.matshow(scores_mat[:,1]);
plt.colorbar();
plt.matshow(scores_mat[:,2]);
plt.colorbar();

In [ ]:
for x in range(canvas.shape[0]):
    for y in range(canvas.shape[1]):
        for z in range(canvas.shape[2]):
            if x != 0 and y != 0 and z != 0:
                p2 = np.dot(T, [x,y,z,1])
                x2,y2,z2 = p2[:3]/p2[-1]
                canvas1[x,y,z] = volume2[int(x2),int(y2),int(z2)]

In [ ]:
canvas1[]